In [1]:
from tensorboard.backend.event_processing import event_accumulator
import pandas as pd
import os
import re
import tensorflow as tf
import numpy as np

2023-04-01 15:51:37.862493: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-01 15:51:37.939417: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def loadtfevent(filepath):
    ea=event_accumulator.EventAccumulator(filepath,size_guidance=event_accumulator.STORE_EVERYTHING_SIZE_GUIDANCE) 
    ea.Reload()
    keys=ea.Tags()['tensors']
    if len(keys)==0:
        return pd.DataFrame()
    tensors=ea.tensors
    df=pd.DataFrame()
    lists=[tensors.Items(key) for key in keys]
    length=len(lists[0])
    for i in range(length):
        datas=[]
        step=lists[0][i].step
        walltime=lists[0][i].wall_time
        for l in lists:
            tmp=tf.make_ndarray(l[i].tensor_proto)
            datas.append(np.nan if tmp.size==0 else tmp)
        datas=np.array(datas)[None,:]
        dfi=pd.DataFrame(datas,columns=keys)
        dfi['step']=step
        dfi['walltime']=walltime
        df=pd.concat([df,dfi],ignore_index=True,)
    return df

In [3]:
def loadevents(log_path):
    log_list=os.listdir(log_path)
    df=pd.DataFrame()
    for log in log_list:
        dfi=loadtfevent(os.path.join(log_path,log))
        df=pd.concat([df,dfi],ignore_index=True,)
    return df
def loaddatas(model_path):
    model_list=os.listdir(model_path)
    regex=f'.*logs.*'
    var_match=re.compile(regex)
    datas={}
    for i,m in enumerate(model_list):
        ret=var_match.match(m)
        if ret:
            datas[m]={}
            if os.path.exists(os.path.join(model_path,m,'train')):
                datas[m]['train']=loadevents(os.path.join(model_path,m,'train'))
            if os.path.exists(os.path.join(model_path,m,'validation')):
                datas[m]['validation']=loadevents(os.path.join(model_path,m,'validation'))
    return datas

In [6]:
model_path = './saved_model'
model_lists=os.listdir(model_path)
for file in ['chief','.ipynb_checkpoints','data.npz',]:
    model_lists.remove(file)
datasets={}
model_lists

['2', '3', '0', 'train', '1', '4']

In [13]:
for model in model_lists:
    print(model)
    datasets[model]=loadtfevent(os.path.join(model_path,model))

unprogressive
ground
ground_progressive
progressive_extended2
progressive_concave
progressive
progressive_convex
progressive_extended


In [14]:
np.save('log_data.npy',datasets)

In [4]:
def subtract_alldata(data_path):
    datas={}
    regex=f'^events\.out\.tfevents\..*\.v2$'
    var_match=re.compile(regex)
    for path,file_dir,files in os.walk(data_path):
        for file in files:
            ret=var_match.match(file)
            if ret:
                print(os.path.join(path,file))
                datas[path]=loadtfevent(os.path.join(path,file))
    return datas

In [5]:
automldata=subtract_alldata('./automl')

./automl/logs/2/execution0/events.out.tfevents.1680102918.SMC.2681602.6.v2
./automl/logs/7/execution0/events.out.tfevents.1680239104.SMC.2681602.20.v2
./automl/logs/7/execution0/validation/events.out.tfevents.1680239574.SMC.2681602.22.v2
./automl/logs/7/execution0/train/events.out.tfevents.1680239113.SMC.2681602.21.v2
./automl/logs/6/execution0/events.out.tfevents.1680211996.SMC.2681602.17.v2
./automl/logs/6/execution0/validation/events.out.tfevents.1680212557.SMC.2681602.19.v2
./automl/logs/6/execution0/train/events.out.tfevents.1680212003.SMC.2681602.18.v2
./automl/logs/5/execution0/events.out.tfevents.1680186895.SMC.2681602.14.v2
./automl/logs/5/execution0/validation/events.out.tfevents.1680187445.SMC.2681602.16.v2
./automl/logs/5/execution0/train/events.out.tfevents.1680186898.SMC.2681602.15.v2
./automl/logs/3/execution0/events.out.tfevents.1680102925.SMC.2681602.7.v2
./automl/logs/3/execution0/validation/events.out.tfevents.1680103769.SMC.2681602.9.v2
./automl/logs/3/execution0/tr

In [6]:
np.save('log_automl.npy',automldata)